In [1]:
import time, os, ssl, time, queue, socket, struct, random, threading
from evdev import InputDevice, ecodes
q = queue.Queue() # cache the command to be send

In [2]:
def heartbeat_daemon():
# Send an invalid action to keep the connection alive
# would be ignored by rover but prevent a timeout for the control link
    while True: 
        q.put(struct.pack('<hi', -1, 0)) 
        time.sleep(3)
threading.Thread(target=heartbeat_daemon).start() 

In [3]:
device = InputDevice('/dev/input/event0')   # Events for Xbox One S Wireless Controller
def controller_daemon(): 
    for event in device.read_loop(): 
        if event.type in [ecodes.EV_ABS, ecodes.EV_KEY]:   
            q.put(struct.pack('<hi', event.code, event.value)) # put it into a queue
threading.Thread(target=controller_daemon).start() 

In [4]:
# Establish connection with server

server_ip = 'guoxiaokang.net'
ctrl_port = 54322 
server_cert = 'secret/server.crt'
client_cert = 'secret/driver.crt'
client_key  = 'secret/driver.key'

context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert)
context.load_cert_chain(certfile=client_cert, keyfile=client_key) 

def control_link_daemon():
    while True: 
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
        ss.settimeout(5)  
        while True:
            try: 
                print('Try to connect the server')
                ss.connect((server_ip, ctrl_port))  
                print('Succeed a connection') 
                break
            except (ConnectionRefusedError, ConnectionResetError):
                print('Failed to connect the server and retry the connection in 5 seconds')
                time.sleep(5)
                continue
        while True:
            try:
                item = q.get()
                #print(struct.unpack('<hi', item))
                ss.send(item) # Send code and value to server  
            except socket.timeout as e: 
                ss.shutdown(socket.SHUT_RDWR)
                ss.close()
                break
            except (ConnectionResetError, BrokenPipeError) as e:
                print('Server ended the connection and connection reestablishment will be retried in 5 seconds:', e)
                time.sleep(5) 
                break
threading.Thread(target=control_link_daemon).start() 

Try to connect the server
Succeed a connection
